In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib agg

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
       
from HPIB.HP4155 import HP4155
from HPIB.HPT import Plot, PlotVgs, PlotVp, CalcIsSat, SecDer, Plot2P, PlotDiode4P,WriteLog
from HPIB.DevParams import UMC
from HPIB.INOSerial import Arduino

from BFmodule import DR

from YFunc import YFuncExtraction
#from Test import TestDevice, WriteLog

from IPython.display import clear_output, display
from os import makedirs, rename
from os.path import isfile
from time import sleep
from datetime import datetime, timedelta

Elsa=DR()

HP=HP4155("GPIB0::17", debug=False)
HP.IntTime="LONG"
HP.reset()

SM=Arduino("COM3")
print(SM.ask('*'.encode()))

Opening API at 143.106.18.130
Elsa v2.4.2
HEWLETT-PACKARD,4155A,0,01.04:01.04:01.00
Comm Er


AttributeError: 'Arduino' object has no attribute 'inst'

In [ ]:
prepath = "C:/Users/Zucchi/Documents/Medidas/250424 Nb/"
makedirs(prepath, exist_ok=True)

In [ ]:
def Measure4P(savepath, I=1e-3, Im='SMU1',Ip='SMU2', Vm='SMU4', Vp='SMU3'):
    global HP, SM, Elsa
    HP.Samp4P(I=I, Im=Im, Ip=Ip, Vm=Vm, Vp=Vp, points=10, interval=3e-3)
    now=datetime.now().strftime('%y%m%d %H%M%S')
            
    name=HP.SingleSave(f"{savepath}/4P {Im[-1]}{Ip[-1]}{Vp[-1]}{Vm[-1]}p - {now}.csv", timeout=30)
    R=[]
    try:
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(name, name.replace('-', f'- {temp} -'))
        name=name.replace('-', f'- {temp} -')
    except Exception as err:
        print (">>> Error:", err)
    df=pd.read_csv(name, header=[0, 1]) 
    R+=[np.average(df['Vf']['None'].to_numpy()/df['If']['None'].to_numpy())]
    
    
    HP.SetSMU(Ip, 'VIp', 'Ip', 'I', 'CONS', Value=-1e-3, Comp=2)
    now=datetime.now().strftime('%y%m%d %H%M%S')
            
    name=HP.SingleSave(f"{savepath}/4P {Im[-1]}{Ip[-1]}{Vp[-1]}{Vm[-1]}m - {now}.csv", timeout=30)
    try:
        temp=format(Elsa.GetT('t4k'), '07.3f')
        rename(name, name.replace('-', f'- {temp} -'))
        name=name.replace('-', f'- {temp} -')
    except Exception as err:
        print (">>> Error:", err)
    df=pd.read_csv(name, header=[0, 1])
    R+=[np.average(df['Vf']['None'].to_numpy()/df['If']['None'].to_numpy())]
    
    return R
    
def MeasureResistivity(savepath, I=1e-3):
    makedirs(savepath, exist_ok=True)
    R=[]
    
    R+=Measure4P(savepath, I=1e-3, Im='SMU1',Ip='SMU2', Vm='SMU4', Vp='SMU3')
    R+=Measure4P(savepath, I=1e-3, Im='SMU2',Ip='SMU3', Vm='SMU1', Vp='SMU4')
    R+=Measure4P(savepath, I=1e-3, Im='SMU3',Ip='SMU4', Vm='SMU2', Vp='SMU1')
    R+=Measure4P(savepath, I=1e-3, Im='SMU4',Ip='SMU1', Vm='SMU3', Vp='SMU2')

    return np.average(R), np.std(R), R

In [ ]:
SM.opench(1)

###### Warmup measurement

MsrNo=1 #??? n tem tipo só uma medida nesse caso?

current_temp=np.around(Elsa.GetT('t4k')) #temperatura da elza 


direction="Warmup"
freq="Fine"
finish_temp=30
freq_temp=1 #A cada quantos kelvin ele vai medir

if not isfile(f"{prepath}/RxT {direction}.log"):
    with open(f"{prepath}/RxT {direction}.log", 'w') as MyFile:
        MyFile.write(f"Temp,ro,R,R1p,R1m,R2p,R2m,R3p,R3m,R4p,R4m\n")

HP.IntTime='LONG' #TEMPO DE INTEGRAÇÃO 
HP.LongNPLC=2   #QUANTOS CICLOS DA REDE ELE VAI ADOTAR COMO TEMPO DE INTEGRAÇÃO (120HZ)

#INTEGRAÇÃO PRA FAZER O VALOR MEDIO DAS MEDIDAS 

clear_output()

prog_bar=display('',display_id=True) #FUNÇÃO DE ATUALIZAÇÃO DO PYTHON, POSSO ATUALIZAR NESSA LINHA A MEDIDA

last_temp=current_temp

while (last_temp <= finish_temp):
    #loop until temp changes and is a multiple of freq_temp
    start=datetime.now()
    while not (np.around(current_temp)!=last_temp):
        #verifica se está na temperatura correta (25 20 15) e coisa e tal ae ele sai do loop
        if (datetime.now()-start).seconds/60 > 120:
            break
        for i in range(12):
            prog_bar.update(f"{datetime.now().strftime('%H:%M:%S')} - T = {format(current_temp, '.1f')} K ({format(last_temp, '.1f')})")
            sleep(1)
    
        current_temp=Elsa.GetT('t4k')

    last_temp=np.around(current_temp)

    #aqui ele ja ta saindo a temperatura na base 5 
    
    for Msr in range(MsrNo): #considerando uma medida por temperatura faça:
        plt.close('all')
        clear_output()
        prog_bar=display('Measuring',display_id=True) #cria uma nova aba de progressão de medida 

        start=datetime.now()
        WriteLog(f"# {current_temp} K - Measurement {Msr+1} - {datetime.now().strftime('%y%m%d %H%M')}\n", f'{prepath}/{direction}/log.txt')
        SM.opench(1)

        RAvg, StDevR, Rarray=MeasureResistivity(f"{prepath}/{direction}/{freq}")

        ro=(np.pi/np.log(2))*100e-9*RAvg

        with open(f"{prepath}/RxT {direction}.log", 'a') as MyFile:
            MyFile.write(f"{format(last_temp, '07.3f')},{format(ro, '.5e')},{format(RAvg, '.5e')},{','.join([format(x, '.5e') for x in Rarray])}\n")

    
        WriteLog(f"{current_temp} K - Measurement {Msr+1} end . Duration: {int((datetime.now()-start).seconds/60)}:{(datetime.now()-start).seconds%60:02d}\n\n#######################\n", f'{prepath}/{direction}/log.txt')

    try:
        current_temp=Elsa.GetT('t4k')
    except:
        pass
        if i < MsrNo:
            start=datetime.now()
            while(datetime.now()-start).seconds < MsrWait*60:
                prog_bar.update(f"Waiting: {int(MsrWait*60)-(datetime.now()-start).seconds} s")
                sleep(1)
prog_bar.update("Ramp measurement done")

In [2]:
HP.close()
INO.close()
print("Comm closed")

NameError: name 'INO' is not defined

In [1]:
current_temp=np.around(Elsa.GetT('t4k')) #temperatura da elza 

HP.IntTime='LONG' #TEMPO DE INTEGRAÇÃO 
HP.LongNPLC=2   #QUANTOS CICLOS DA REDE ELE VAI ADOTAR COMO TEMPO DE INTEGRAÇÃO (120HZ)

#INTEGRAÇÃO PRA FAZER O VALOR MEDIO DAS MEDIDAS 
if(current_temp<20){
    #warmup
    last_temperature = 200
    inc = 1
}else{
    #cooldown
    last_temperature = 5
    inc = 0
}

while(abs(last_temperature - current_temperature) > 2):
    temperature_conf(10,1)
    




SyntaxError: invalid syntax (2528260288.py, line 10)

In [17]:


def temperature_conf(step_,step_fine):
    current_temp = np.around(Elsa.GetT('t4k'))
    last_temp = current_temp
    if(current_temp > 30):
        step = step_
    else:
        step = step_fine
    while (current_temp == last_temp or ((current_temp % step)!= 0)):
        last_temp = current_temp
        current_temp = np.around(Elsa.GetT('t4k'))
        sleep(1)
    print(current_temp)

temperature_conf(10,1)


NameError: name 'Elsa' is not defined

In [12]:
from time import sleep 
from IPython.display import clear_output, display

s=input("Only one measure?(y/n) ")
if(s=='y'):
    print("show")
if(s=='n'):
    print("e isso ae")
sleep(3)
clear_output()

In [ ]:
    def SingleSave(self, path=".", timeout=2, real=False):
        if self.term=="0": return "Parameters not set"
        
        print(f"Measuring {self.term} ", end='')
        self.measure()
        
        Poll=self.PollDR(1, 1, timeout)
        
        if Poll:
            return "Operation stopped or timeout"
        if real:
            df=self.get_realdata()
        else:
            df=self.get_data()